**Kushal Barot (202318006)** <br><br> BD Assignment-1 <br>Task-B,C

Task B

In [ ]:
import requests

url = "https://real-time-quotes1.p.rapidapi.com/api/v1/symbol/stock"

querystring = {"includeNames":"true"}

headers = {
	"X-RapidAPI-Key": "86ceb7348amsh415e7086b519ed2p183b66jsn4ee190304a3d",
	"X-RapidAPI-Host": "real-time-quotes1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# print(response.json())
for i in response.json():
  print(i)

Streaming output truncated to the last 5000 lines.
{'symbol': '7988.T', 'name': 'Nifco Inc.'}
{'symbol': 'ALFWQ', 'name': 'Alfi, Inc.'}
{'symbol': 'TFF.PA', 'name': 'TFF Group'}
{'symbol': 'PRZM', 'name': 'Prism Technologies Group, Inc.'}
{'symbol': 'XTG.TO', 'name': 'Xtra-Gold Resources Corp.'}
{'symbol': 'IAUX', 'name': 'i-80 Gold Corp.'}
{'symbol': '8544.T', 'name': 'The Keiyo Bank, Ltd.'}
{'symbol': '2360.HK', 'name': 'Best Mart 360 Holdings Limited'}
{'symbol': '075580.KS', 'name': 'Sejin Heavy Industries Co., Ltd.'}
{'symbol': 'EMA-PC.TO', 'name': 'Emera Incorporated'}
{'symbol': 'PHYT-UN', 'name': 'Pyrophyte Acquisition Corp.'}
{'symbol': 'BAIBDA.CO', 'name': 'Bæredygtige Klimaaktier Akk. Kl'}
{'symbol': '014830.KS', 'name': 'UNID Company Ltd.'}
{'symbol': 'COFB.BR', 'name': 'Cofinimmo SA'}
{'symbol': 'DR.TO', 'name': 'Medical Facilities Corporation'}
{'symbol': 'EMG.L', 'name': 'Man Group Limited'}
{'symbol': 'NXGL', 'name': 'NEXGEL, Inc.'}
{'symbol': 'BCDMF', 'name': 'Kuuhubb 

In [ ]:
import pandas as pd

df = pd.json_normalize(response.json())

file = "output.csv"

df.to_csv(file,index="False")

Task C

In [ ]:
!pip install cloudscraper

In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_page(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    data = {
        'Product_Name': [],
        'rating': [],
        'review': [],
        'price': [],
        'mrp': []
    }

    levies = soup.find_all('div', class_='puis-card-container s-card-container s-overflow-hidden aok-relative puis puis-v1x9xtdrqwkf2b2sy3gv3bhvhpn s-latency-cf-section puis-card-border')

    for brand in levies:
        name = brand.find('h2').text.strip()
        rating = brand.find('span', class_='a-icon-alt').text.strip() if brand.find('span', class_='a-icon-alt') else 'N/A'
        reviews = brand.find('span', class_='a-size-base s-underline-text').text.strip() if brand.find('span', class_='a-size-base s-underline-text') else 'N/A'
        price = brand.find('span', class_='a-price-whole').text.strip() if brand.find('span', class_='a-price-whole') else 'N/A'
        mrp = brand.find('span', class_='a-offscreen').text.strip() if brand.find('span', class_='a-offscreen') else 'N/A'


        if all([name, rating, reviews, price]):
            data['Product_Name'].append(name)
            data['rating'].append(rating)
            data['review'].append(reviews)
            data['price'].append(price)
            data['mrp'].append(mrp)

    return pd.DataFrame(data)


page_urls = [
    'https://www.amazon.in/s?k=levis&rh=n%3A1968024031&ref=nb_sb_noss',
    'https://www.amazon.in/s?k=levis&i=apparel&rh=n%3A1968024031&page=2&qid=1706682683&ref=sr_pg_2',
    'https://www.amazon.in/s?k=levis&i=apparel&rh=n%3A1968024031&page=3&qid=1706682700&ref=sr_pg_3',
    'https://www.amazon.in/s?k=levis&i=apparel&rh=n%3A1968024031&page=3&qid=1706682700&ref=sr_pg_3',
    'https://www.amazon.in/s?k=levis&i=apparel&rh=n%3A1968024031&page=5&qid=1706682739&ref=sr_pg_5',
    'https://www.amazon.in/s?k=levis&i=apparel&rh=n%3A1968024031&page=6&qid=1706682755&ref=sr_pg_6',
    'https://www.amazon.in/s?k=levis&i=apparel&rh=n%3A1968024031&page=7&qid=1706682779&ref=sr_pg_7',
]

# Scraping data from multiple pages
dfs = []
for url in page_urls:
    df = scrape_page(url)
    dfs.append(df)
    time.sleep(1)

# Concatenate the dataframes from each page
result_df = pd.concat(dfs, ignore_index=True)


try:
    old_data = pd.read_csv('scraped_data_multiple.csv')
except FileNotFoundError:
    old_data = pd.DataFrame()


final_df = pd.concat([old_data, result_df], ignore_index=True)


final_df.to_csv('scraped_data_multiple.csv', index=False, encoding='utf-8-sig')

print(final_df)
